In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from collections import Counter

In [2]:
#mainDF = pd.read_excel("Data//September_09//LabHoursData.xlsx")
mainDF = pd.read_excel("Data//December_12//LabHoursData.xlsx")
mainDF.columns

Index(['#', 'Machine Type', 'Machine', 'Wing', 'Floor', 'Run No', 'Pcs',
       'StartDate', 'StartTime', 'StopDate', 'StopTime', 'Run %',
       'LateStartReason', 'Stop Reasons', 'Recipe', 'Run Intensity',
       'Ini Category', 'Grown Category', 'RunStatus', 'IniAvgHeight',
       'ProdHeight', 'GrownHeight', 'GrowingHours', 'BreakdownTime',
       'From To Hours', 'Actual Hours', 'RunHours', 'IniWeight', 'FinalWeight',
       'GrownWeight', 'Width', 'Length', 'Dmtr', 'Unit', 'Month', 'GrRate'],
      dtype='object')

In [3]:
#remove = "BreakdownTime", "GrowingHours"
final_DF = mainDF[["Machine", "GrownWeight", "Dmtr", "GrowingHours", "GrownHeight",
                   "Actual Hours", "GrRate", "Stop Reasons", "LateStartReason"]]
#final_DF[final_DF.isna().any(axis=1)]
final_DF = final_DF.rename(columns = {'Stop Reasons':'Stop_Reasons', 'Actual Hours':'Actual_Hours'})

final_DF.tail(2)

,Machine,GrownWeight,Dmtr,GrowingHours,GrownHeight,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
4834,AG68,21.10723,55.69,48.98,0.48931,4.12,9.99,",",HLD CLN
4835,NaN,NaN,NaN,"Sum: 577,633.49",NaN,NaN,Avg: 12.63,NaN,NaN


In [4]:
nullID = final_DF[final_DF[['Machine','GrownWeight', 'GrownHeight','Dmtr', \
                            'Actual_Hours', 'GrRate']].isna().any(axis=1)].index
nullID

Int64Index([4835], dtype='int64')

In [5]:
final_DF = final_DF.drop(nullID)

In [6]:
final_DF.head(2)

,Machine,GrownWeight,Dmtr,GrowingHours,GrownHeight,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
0,CC22,6.263735,51.17,11.07,0.15281,186.24,13.89,"TORRENT POWER CUT , N/A",ON TIME
1,CC22,89.995000,41.07,235,3.14000,235.00,13.36,"TIDE O , N/A",ON TIME


In [7]:
def count_words(df_temp,column_name):
    df_temp[column_name] = df_temp[column_name].str.rstrip('')
    df_temp[column_name] = df_temp[column_name].str.lstrip('')

    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(',,',',')

    df_temp[column_name] = df_temp[column_name].str.replace('\' ','\'')
    df_temp[column_name] = df_temp[column_name].str.replace(' \'','\'')

    df_temp[column_name] = df_temp[column_name].str.rstrip(',')
    df_temp[column_name] = df_temp[column_name].str.lstrip(',')

    df_temp[column_name] = df_temp[column_name].str.split(',')
    df_temp[column_name] = df_temp[column_name].apply(lambda x: Counter(x))
    
    df_temp[column_name] = df_temp[column_name].apply(lambda x: dict(x))
    
    return df_temp

In [8]:
#join all string values of Stop_Reasons in groupby
df_Stop_reason = final_DF.groupby(['Machine'])['Stop_Reasons'].agg(lambda x: ','.join(x.dropna())).reset_index()

#join all string values of LateStartReason in groupby
df_LateStartReason = final_DF.groupby(['Machine'])['LateStartReason'].agg(lambda x: ','.join(x.dropna())).reset_index()

In [9]:
df_LateStartReason

,Machine,LateStartReason
0,AA1,"STG,CHL,ON TIME,HLD CLN,HLD CLN,HLM LT,ON TIME"
1,AA10,"MW,CHL,ON TIME,ON TIME,ON TIME"
2,AA11,"ON TIME,CHL,ON TIME,HLD CLN,ON TIME,HLD CLN"
3,AA12,"HLD CLN,CHL,ON TIME,ON TIME,STG,ON TIME"
4,AA13,"CHMB CHNG,CHL,UPS"
...,...,...
994,CC16,"ON TIME,ON TIME,ON TIME,ON TIME,ON TIME,PMP CHNG"
995,CC17,"ON TIME,ELECTRICITY,ON TIME,ON TIME,ON TIME"
996,CC20,"ON TIME,ON TIME,ON TIME,NL T,ON TIME,ON TIME"
997,CC21,"ON TIME,ELECTRICITY,ON TIME,ON TIME,ON TIME"


In [10]:
final_DF.columns

Index(['Machine', 'GrownWeight', 'Dmtr', 'GrowingHours', 'GrownHeight',
       'Actual_Hours', 'GrRate', 'Stop_Reasons', 'LateStartReason'],
      dtype='object')

In [11]:
df_Stop_reason['Stop_Reasons_counts'] = df_Stop_reason['Stop_Reasons']
df_Stop_reason = count_words(df_Stop_reason,'Stop_Reasons_counts')

df_LateStartReason['LateStartReason_counts'] = df_LateStartReason['LateStartReason']
df_LateStartReason = count_words(df_LateStartReason,'LateStartReason_counts')

In [12]:
df_LateStartReason

,Machine,LateStartReason,LateStartReason_counts
0,AA1,"STG,CHL,ON TIME,HLD CLN,HLD CLN,HLM LT,ON TIME","{'STG': 1, 'CHL': 1, 'ON TIME': 2, 'HLD CLN': ..."
1,AA10,"MW,CHL,ON TIME,ON TIME,ON TIME","{'MW': 1, 'CHL': 1, 'ON TIME': 3}"
2,AA11,"ON TIME,CHL,ON TIME,HLD CLN,ON TIME,HLD CLN","{'ON TIME': 3, 'CHL': 1, 'HLD CLN': 2}"
3,AA12,"HLD CLN,CHL,ON TIME,ON TIME,STG,ON TIME","{'HLD CLN': 1, 'CHL': 1, 'ON TIME': 3, 'STG': 1}"
4,AA13,"CHMB CHNG,CHL,UPS","{'CHMB CHNG': 1, 'CHL': 1, 'UPS': 1}"
...,...,...,...
994,CC16,"ON TIME,ON TIME,ON TIME,ON TIME,ON TIME,PMP CHNG","{'ON TIME': 5, 'PMP CHNG': 1}"
995,CC17,"ON TIME,ELECTRICITY,ON TIME,ON TIME,ON TIME","{'ON TIME': 4, 'ELECTRICITY': 1}"
996,CC20,"ON TIME,ON TIME,ON TIME,NL T,ON TIME,ON TIME","{'ON TIME': 5, 'NL T': 1}"
997,CC21,"ON TIME,ELECTRICITY,ON TIME,ON TIME,ON TIME","{'ON TIME': 4, 'ELECTRICITY': 1}"


In [13]:
final_DF.GrownWeight = pd.to_numeric(final_DF.GrownWeight)
final_DF.GrowingHours = pd.to_numeric(final_DF.GrowingHours)
final_DF.ProdHeight = pd.to_numeric(final_DF.GrownHeight)
final_DF.Dmtr = pd.to_numeric(final_DF.Dmtr)
final_DF.Actual_Hours = pd.to_numeric(final_DF.Actual_Hours)
final_DF.GrRate = pd.to_numeric(final_DF.GrRate)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10188\2619673649.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  final_DF.ProdHeight = pd.to_numeric(final_DF.GrownHeight)


In [14]:
final_DF.groupby(['Machine'])['GrRate', 'Actual_Hours','Dmtr'].mean().reset_index()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10188\1685998223.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine'])['GrRate', 'Actual_Hours','Dmtr'].mean().reset_index()


,Machine,GrRate,Actual_Hours,Dmtr
0,AA1,12.780357,118.485714,53.024286
1,AA10,12.186000,169.070000,54.808000
2,AA11,13.031667,119.768333,54.466667
3,AA12,12.260000,92.191667,53.940000
4,AA13,11.616667,45.500000,31.333333
...,...,...,...,...
994,CC16,14.617917,120.080000,40.833333
995,CC17,14.432500,140.170000,41.830000
996,CC20,13.796667,132.826667,41.218333
997,CC21,14.515000,141.560000,41.820000


In [15]:
df_machine_less35 = pd.merge(
        final_DF.groupby(['Machine'])['GrownWeight','GrowingHours','GrownHeight'].sum().reset_index(),
        final_DF.groupby(['Machine'])['GrRate', 'Actual_Hours','Dmtr'].mean().reset_index(), 
                                      how="left", on=["Machine"])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10188\4017109264.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine'])['GrownWeight','GrowingHours','GrownHeight'].sum().reset_index(),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10188\4017109264.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine'])['GrRate', 'Actual_Hours','Dmtr'].mean().reset_index(),


In [16]:
df_machine_less35 = df_machine_less35.rename(columns={'GrownWeight':'GrownWeight_sum','GrowingHours':'GrowingHours_sum','GrownHeight':'GrownHeight_sum',
                                                      'GrRate':'GrRate_avg', 'Actual_Hours':'Actual_Hours_avg','Dmtr':'Dmtr_avg'})

In [17]:
df_machine_less35.head(2)

,Machine,GrownWeight_sum,GrowingHours_sum,GrownHeight_sum,GrRate_avg,Actual_Hours_avg,Dmtr_avg
0,AA1,356.41137,615.82,8.389494,12.780357,118.485714,53.024286
1,AA10,371.21194,650.56,8.280800,12.186000,169.070000,54.808000


In [18]:
# df_machine_less35.index.name = "ID"
# df_machine_less35.reset_index(inplace=True)

In [19]:
df_machine_less35.head(2)

,Machine,GrownWeight_sum,GrowingHours_sum,GrownHeight_sum,GrRate_avg,Actual_Hours_avg,Dmtr_avg
0,AA1,356.41137,615.82,8.389494,12.780357,118.485714,53.024286
1,AA10,371.21194,650.56,8.280800,12.186000,169.070000,54.808000


In [20]:
df_machine_less35["GrownWeight_less_35"] = df_machine_less35['GrownWeight_sum'] - (df_machine_less35["GrownWeight_sum"] * 35 / 100)
# df_machine_less35["GrRate_rank"] = df_machine_less35['GrRate_avg'].apply(lambda x: 1 if (x>12.50 and x<=13.00 ) else 0)

In [21]:
#=============================================================================

In [22]:
#==============================================================================

In [23]:
def add_sorting_rank_to_column(temp_df, column_name, ascending_order):
    temp_df[column_name] = pd.to_numeric(temp_df[column_name])
    temp_df = temp_df.sort_values(by=[column_name], ascending=ascending_order).reset_index()
    temp_df.index.name = column_name + "_rank"
    temp_df = temp_df.reset_index()
    temp_df[column_name + "_rank"] += 1
    temp_df.index.name = "delete_" + column_name
    return temp_df

In [24]:
df_machine_less35

,Machine,GrownWeight_sum,GrowingHours_sum,GrownHeight_sum,GrRate_avg,Actual_Hours_avg,Dmtr_avg,GrownWeight_less_35
0,AA1,356.411370,615.82,8.389494,12.780357,118.485714,53.024286,231.667391
1,AA10,371.211940,650.56,8.280800,12.186000,169.070000,54.808000,241.287761
2,AA11,341.860978,622.23,8.001637,13.031667,119.768333,54.466667,222.209636
3,AA12,338.867586,650.48,7.788574,12.260000,92.191667,53.940000,220.263931
4,AA13,44.620262,126.65,1.551827,11.616667,45.500000,31.333333,29.003170
...,...,...,...,...,...,...,...,...
994,CC16,247.479880,673.92,9.672787,14.617917,120.080000,40.833333,160.861922
995,CC17,283.297424,682.84,9.603078,14.432500,140.170000,41.830000,184.143326
996,CC20,256.739000,671.74,9.781118,13.796667,132.826667,41.218333,166.880350
997,CC21,260.665896,679.81,10.088057,14.515000,141.560000,41.820000,169.432832


In [25]:
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "GrownWeight_sum", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "GrowingHours_sum", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "GrownHeight_sum", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "Actual_Hours_avg", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "Dmtr_avg", False)
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35 = df_machine_less35.drop(["delete_Dmtr_avg","delete_GrownHeight_sum","delete_Actual_Hours_avg","delete_GrownWeight_sum","delete_GrowingHours_sum"], axis=1)

In [26]:
df_machine_less35

,Dmtr_avg_rank,Actual_Hours_avg_rank,GrownHeight_sum_rank,GrowingHours_sum_rank,GrownWeight_sum_rank,index,Machine,GrownWeight_sum,GrowingHours_sum,GrownHeight_sum,GrRate_avg,Actual_Hours_avg,Dmtr_avg,GrownWeight_less_35
0,1,932,85,109,1,161,AC23,554.627005,648.73,8.922218,14.002083,77.896667,66.081667,360.507553
1,2,410,742,881,11,282,AE29,416.215000,524.17,7.020000,14.590000,131.025000,63.765000,270.539750
2,3,780,554,876,3,310,AE56,470.786580,529.06,7.542792,14.808214,101.288571,59.675714,306.011277
3,4,993,992,992,988,343,AF38,17.279176,29.82,0.391163,13.117500,1.100000,58.640000,11.231464
4,5,943,983,985,977,345,AF4,50.325000,75.75,1.260000,16.630000,75.450000,56.340000,32.711250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,995,983,981,982,986,26,AA34,33.866783,94.98,1.308325,15.421250,14.400000,36.275000,22.013409
995,996,687,953,947,968,414,AG41,66.709194,240.58,2.680990,11.630000,109.933333,33.946667,43.360976
996,997,975,976,974,980,4,AA13,44.620262,126.65,1.551827,11.616667,45.500000,31.333333,29.003170
997,998,984,987,989,994,102,AB39,8.303456,59.98,1.008601,15.693333,12.923333,30.760000,5.397246


In [27]:
df_machine_less35["GrRate_class"] = df_machine_less35['GrRate_avg'].apply(lambda x: 'A' if (x>12.50 and x<=13.00 ) else 'B' if x<=12.50 else 'C')
df_machine_less35.index.name = "delete_GrRate_rank"
order = {'A': True, 'B': False, 'C': True}
df_machine_less35 = (df_machine_less35.groupby('GrRate_class', group_keys=False)
   .apply(lambda s: s.sort_values(by='GrRate_avg', ascending=order[s.name]))
).reset_index()

df_machine_less35.index.name = "GrRate_rank"
df_machine_less35 = df_machine_less35.reset_index()
k = len(df_machine_less35[df_machine_less35.GrRate_class == "A"])
df_machine_less35['GrRate_rank'] = df_machine_less35['GrRate_rank'].apply(lambda x: 1 if x<k else x)
df_machine_less35['GrRate_rank'] = df_machine_less35['GrRate_rank'].apply(lambda x: x-k+2 if x>=k else x)
df_machine_less35.drop(columns=["delete_GrRate_rank"], inplace=True)

In [28]:
pd.set_option("display.max_rows", None)
df_machine_less35[["GrRate_avg","GrRate_class","GrRate_rank"]]

,GrRate_avg,GrRate_class,GrRate_rank
0,12.504000,A,1
1,12.505625,A,1
2,12.506875,A,1
3,12.507000,A,1
4,12.509000,A,1
5,12.517500,A,1
6,12.518750,A,1
7,12.520000,A,1
8,12.534375,A,1
9,12.537500,A,1


In [29]:
df_machine_less35['Total_rank'] = df_machine_less35['GrownWeight_sum_rank'] + df_machine_less35['Dmtr_avg_rank'] + \
    df_machine_less35['GrowingHours_sum_rank'] + df_machine_less35['Actual_Hours_avg_rank'] + \
    df_machine_less35["GrownHeight_sum_rank"] + df_machine_less35['GrRate_rank']

In [30]:
df_results = pd.merge(pd.merge(df_machine_less35,df_Stop_reason,how="left", on=["Machine"]),df_LateStartReason,how="left", on=["Machine"])

In [31]:
def rangeApply(k):
    if k%1 <= .50:
        return (str(int(k)+.00) +'-'+ str(int(k)+.50))
    else:
        return (str(int(k)+.50) +'-'+ str(int(k)+1.00))

In [32]:
df_results["GrRate_Range"] = df_results.GrRate_avg.apply(rangeApply)
df_results[['GrRate_avg', "GrRate_Range"]]

,GrRate_avg,GrRate_Range
0,12.504000,12.5-13.0
1,12.505625,12.5-13.0
2,12.506875,12.5-13.0
3,12.507000,12.5-13.0
4,12.509000,12.5-13.0
5,12.517500,12.5-13.0
6,12.518750,12.5-13.0
7,12.520000,12.5-13.0
8,12.534375,12.5-13.0
9,12.537500,12.5-13.0


In [33]:
df_results.columns

Index(['GrRate_rank', 'Dmtr_avg_rank', 'Actual_Hours_avg_rank',
       'GrownHeight_sum_rank', 'GrowingHours_sum_rank', 'GrownWeight_sum_rank',
       'index', 'Machine', 'GrownWeight_sum', 'GrowingHours_sum',
       'GrownHeight_sum', 'GrRate_avg', 'Actual_Hours_avg', 'Dmtr_avg',
       'GrownWeight_less_35', 'GrRate_class', 'Total_rank', 'Stop_Reasons',
       'Stop_Reasons_counts', 'LateStartReason', 'LateStartReason_counts',
       'GrRate_Range'],
      dtype='object')

In [34]:
df_results = df_results[['index','Machine','GrownWeight_sum','GrownWeight_less_35','Dmtr_avg','Actual_Hours_avg', 
                         'GrowingHours_sum','GrownHeight_sum','GrRate_avg','GrRate_Range','GrRate_rank','GrownWeight_sum_rank',
                         'Actual_Hours_avg_rank', 'GrowingHours_sum_rank','GrownHeight_sum_rank','Dmtr_avg_rank','Total_rank']]#,'Stop_Reasons_counts', 'LateStartReason_counts']]

In [35]:
df_results

,index,Machine,GrownWeight_sum,GrownWeight_less_35,Dmtr_avg,Actual_Hours_avg,GrowingHours_sum,GrownHeight_sum,GrRate_avg,GrRate_Range,GrRate_rank,GrownWeight_sum_rank,Actual_Hours_avg_rank,GrowingHours_sum_rank,GrownHeight_sum_rank,Dmtr_avg_rank,Total_rank
0,550,BAC18,236.817017,153.931061,43.458000,161.324000,601.82,7.345263,12.504000,12.5-13.0,1,745,119,635,640,816,2956
1,492,BAA44,271.534700,176.497555,44.542500,165.625000,602.15,7.766033,12.505625,12.5-13.0,1,512,94,631,450,714,2402
2,882,BBD5,263.486500,171.266225,44.047500,180.950000,633.65,8.067126,12.506875,12.5-13.0,1,558,45,222,325,762,1913
3,176,AC38,354.444515,230.388935,51.816000,109.410000,659.23,7.976244,12.507000,12.5-13.0,1,164,693,56,358,275,1547
4,276,AE22,323.748647,210.436621,53.168000,100.318000,595.69,7.291357,12.509000,12.5-13.0,1,296,796,689,664,152,2598
5,721,BBA12,248.974024,161.833116,42.142500,184.815000,635.98,8.136556,12.517500,12.5-13.0,1,664,37,208,298,913,2121
6,669,BAE5,301.956238,196.271555,46.660000,147.212500,614.90,7.743655,12.518750,12.5-13.0,1,356,229,459,461,506,2012
7,424,AG50,295.933785,192.356960,52.925714,98.112857,576.83,6.853760,12.520000,12.5-13.0,1,379,818,801,799,175,2973
8,468,BAA2,225.759826,146.743887,41.325000,154.690000,634.82,8.473562,12.534375,12.5-13.0,1,831,165,218,177,940,2332
9,897,BBE19,314.875265,204.668922,48.872500,168.687500,598.42,7.595890,12.537500,12.5-13.0,1,316,82,663,530,350,1942


In [36]:
df_results = df_results.sort_values(by=['index'])
df_results = df_results.set_index("index")
if df_results.index[0] == 0:
    df_results.index += 1

In [37]:
df_results.head()

,Machine,GrownWeight_sum,GrownWeight_less_35,Dmtr_avg,Actual_Hours_avg,GrowingHours_sum,GrownHeight_sum,GrRate_avg,GrRate_Range,GrRate_rank,GrownWeight_sum_rank,Actual_Hours_avg_rank,GrowingHours_sum_rank,GrownHeight_sum_rank,Dmtr_avg_rank,Total_rank
index,,,,,,,,,,,,,,,,
1,AA1,356.411370,231.667391,53.024286,118.485714,615.82,8.389494,12.780357,12.5-13.0,1,155,586,443,205,162,1552
2,AA10,371.211940,241.287761,54.808000,169.070000,650.56,8.280800,12.186000,12.0-12.5,97,99,80,97,241,20,634
3,AA11,341.860978,222.209636,54.466667,119.768333,622.23,8.001637,13.031667,13.0-13.5,527,229,568,350,345,35,2054
4,AA12,338.867586,220.263931,53.940000,92.191667,650.48,7.788574,12.260000,12.0-12.5,81,238,864,99,437,72,1791
5,AA13,44.620262,29.003170,31.333333,45.500000,126.65,1.551827,11.616667,11.5-12.0,246,980,975,974,976,997,5148


In [38]:
if os.path.exists("quantityReport_for_only_machine.xlsx"):
    os.remove("quantityReport_for_only_machine.xlsx")
    
df_results.to_excel("quantityReport_for_only_machine.xlsx", header=1, index_label="ID")

In [39]:
df_results.Machine.nunique()

999

In [40]:
df_results

,Machine,GrownWeight_sum,GrownWeight_less_35,Dmtr_avg,Actual_Hours_avg,GrowingHours_sum,GrownHeight_sum,GrRate_avg,GrRate_Range,GrRate_rank,GrownWeight_sum_rank,Actual_Hours_avg_rank,GrowingHours_sum_rank,GrownHeight_sum_rank,Dmtr_avg_rank,Total_rank
index,,,,,,,,,,,,,,,,
1,AA1,356.411370,231.667391,53.024286,118.485714,615.82,8.389494,12.780357,12.5-13.0,1,155,586,443,205,162,1552
2,AA10,371.211940,241.287761,54.808000,169.070000,650.56,8.280800,12.186000,12.0-12.5,97,99,80,97,241,20,634
3,AA11,341.860978,222.209636,54.466667,119.768333,622.23,8.001637,13.031667,13.0-13.5,527,229,568,350,345,35,2054
4,AA12,338.867586,220.263931,53.940000,92.191667,650.48,7.788574,12.260000,12.0-12.5,81,238,864,99,437,72,1791
5,AA13,44.620262,29.003170,31.333333,45.500000,126.65,1.551827,11.616667,11.5-12.0,246,980,975,974,976,997,5148
6,AA14,413.022265,268.464472,54.787500,144.590000,678.48,9.068242,13.356875,13.0-13.5,590,13,263,23,67,21,977
7,AA15,359.935170,233.957861,53.661667,119.956667,657.31,8.198629,12.161667,12.0-12.5,104,138,563,65,270,102,1242
8,AA16,396.724380,257.870847,52.502500,140.225000,674.48,9.533224,14.086250,14.0-14.5,740,34,316,29,31,226,1376
9,AA17,357.901543,232.636003,54.110000,121.114000,661.73,8.294723,12.430500,12.0-12.5,29,147,550,50,235,56,1067


In [41]:
m = 'AA31'
final_DF[(final_DF.Machine==m)]#&(final_DF.Machine_Type==t)]

,Machine,GrownWeight,Dmtr,GrowingHours,GrownHeight,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
3565,AA31,4.605300,45.59,11.08,0.151360,64.15,13.7000,"TORRENT POWER CUT , N/A",ELECTRICITY
3566,AA31,101.990000,53.43,184.00,2.320000,184.00,12.5500,"HEAT , N/A",CHL
3567,AA31,119.760000,54.23,202.00,2.620000,202.00,12.9700,"HEAT , N/A",STG
3568,AA31,100.580000,51.47,193.00,2.400000,193.00,12.4400,"HEAT , N/A",ON TIME
3569,AA31,33.517265,54.27,68.98,0.878977,2.50,12.7425,",",ON TIME


In [42]:
print(final_DF[(final_DF.Machine==m)].GrownWeight.sum())
print(final_DF[(final_DF.Machine==m)].GrowingHours.sum())
print(final_DF[(final_DF.Machine==m)].Actual_Hours.mean())
print(final_DF[(final_DF.Machine==m)].GrownHeight.sum())
print(final_DF[(final_DF.Machine==m)].Dmtr.mean())
print(final_DF[(final_DF.Machine==m)].GrRate.mean())

360.452565
659.0600000000001
129.13
8.370337000000001
51.798
12.880500000000001
